In [1]:
import pandas as pd
import numpy as np
from talib.abstract import *
from sklearn.preprocessing import StandardScaler
from autogluon.tabular import TabularDataset, TabularPredictor

/Users/zhangjiahao/opt/miniconda3/envs/condaVenv39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TRAIN_MARKET_PATH = 'kaggle/input/hku-qids-2023-quantitative-investment-competition/first_round_train_market_data.csv'
TRAIN_FUNADMENTAL_PATH = 'kaggle/input/hku-qids-2023-quantitative-investment-competition/first_round_train_fundamental_data.csv'
TRAIN_RETURN_PATH = 'kaggle/input/hku-qids-2023-quantitative-investment-competition/first_round_train_return_data.csv'


TEST_MARKET_PATH = 'kaggle/input/hku-qids-2023-quantitative-investment-competition/qids_package/first_round_test_market_data.csv'
TEST_FUNADMENTAL_PATH = 'kaggle/input/hku-qids-2023-quantitative-investment-competition/qids_package/first_round_test_fundamental_data.csv'

In [3]:
df_train_market = pd.read_csv(TRAIN_MARKET_PATH)
df_train_return = pd.read_csv(TRAIN_RETURN_PATH)
df_train_fundamental = pd.read_csv(TRAIN_FUNADMENTAL_PATH)

df_test_market = pd.read_csv(TEST_MARKET_PATH)
df_test_fundamental = pd.read_csv(TEST_FUNADMENTAL_PATH)

In [4]:
def split_time(x):
    df1 = x['date_time'].str.split('d', expand=True)
    df1.columns=['code','s']
    code = df1['code']
    df1 = df1['s'].str.split('p', expand=True)
    df1.columns=['day','time_step']
    df2 = x['date_time'].str.rsplit('p', expand=True)
    df2.columns=['day_s','s']
    df1['day_s'] = df2['day_s']
    df1['code'] = code
    x = pd.concat([x,df1],axis=1)
    
    return x

df_train_split = split_time(df_train_market)
df_train = pd.merge(df_train_fundamental,df_train_split, left_on='date_time',right_on='day_s')  
df_train = pd.merge(df_train,df_train_return, left_on='day_s',right_on='date_time')  

df_test_split = split_time(df_test_market)
df_test = pd.merge(df_test_fundamental,df_test_split, left_on='date_time',right_on='day_s')

In [5]:
df_train[['day','time_step']]=df_train[['day','time_step']].apply(pd.to_numeric)
df_test[['day','time_step']]=df_test[['day','time_step']].apply(pd.to_numeric)

In [6]:
def calculate_index(x,df,inputs):
    y=eval(x)
    df[x]=0.0
    df[x]=y(inputs)

In [7]:
assets_final_return=[]
for i in range(0,54): 
    train_data=pd.DataFrame()
    test_data=pd.DataFrame()

    train_data= df_train[df_train['code']=='s{}'.format(i)].groupby(['day']).mean()
    test_data= df_test[df_test['code']=='s{}'.format(i)].groupby(['day']).mean()

    inputs_train = {
    'open':train_data["open"].to_numpy(),
    'high':train_data["high"].to_numpy(),
    'low': train_data["low"].to_numpy(),
    'close':train_data["close"].to_numpy(),
    'volume':train_data["volume"].to_numpy()
    }

    inputs_test = {
        'open':test_data["open"].to_numpy(),
        'high':test_data["high"].to_numpy(),
        'low': test_data["low"].to_numpy(),
        'close':test_data["close"].to_numpy(),
        'volume':test_data["volume"].to_numpy()
    }

    columns_index=['AD','OBV']
    for j in columns_index:
        calculate_index(j,train_data,inputs_train)
        calculate_index(j,test_data,inputs_test)
    
    #StandardScaler
    scaled_features_train = StandardScaler().fit_transform(train_data.values)
    scaled_features_df_train = pd.DataFrame(scaled_features_train, index=train_data.index, columns=train_data.columns)
    scaled_features_df_train['return']=train_data['return']

    scaled_features_test = StandardScaler().fit_transform(test_data.values)
    scaled_features_df_test = pd.DataFrame(scaled_features_test, index=test_data.index, columns=test_data.columns)

    #select feature
    train_data_final=scaled_features_df_train[['turnoverRatio','pe_ttm','pb','ps','pcf','close','volume','AD','OBV','return']]
    test_data_final=scaled_features_df_test[['turnoverRatio','pe_ttm','pb','ps','pcf','close','volume','AD','OBV']]

    #Autogluon
    train_data_tab = TabularDataset(train_data_final)
    label = 'return'
    # fit the data
    predictor = TabularPredictor(label=label).fit( train_data_tab, num_bag_folds=3)

    test_data_tab = TabularDataset(test_data_final)
    preds = predictor.predict(test_data_tab)

    assets_final_return.append(preds)

/var/folders/zn/2h_jn9152knbzpbwpq843zr80000gn/T/ipykernel_2730/3352962815.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train_data= df_train[df_train['code']=='s{}'.format(i)].groupby(['day']).mean()
/var/folders/zn/2h_jn9152knbzpbwpq843zr80000gn/T/ipykernel_2730/3352962815.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_data= df_test[df_test['code']=='s{}'.format(i)].groupby(['day']).mean()
No path specified. Models will be saved in: "AutogluonModels/ag-20230310_210341/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230310_210341/"
AutoGluon Version:  

[1000]	valid_set's rmse: 0.0427615


	-0.0393	 = Validation score   (-root_mean_squared_error)
	12.31s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0397	 = Validation score   (-root_mean_squared_error)
	11.01s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-0.0387	 = Validation score   (-root_mean_squared_error)
	0.41s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0397	 = Validation score   (-root_mean_squared_error)
	2.03s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-0.0374	 = Validation score   (-root_mean_squared_error)
	0.28s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting 

[1000]	valid_set's rmse: 0.0443367


	-0.0391	 = Validation score   (-root_mean_squared_error)
	6.78s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0388	 = Validation score   (-root_mean_squared_error)
	7.17s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-0.0382	 = Validation score   (-root_mean_squared_error)
	0.35s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0389	 = Validation score   (-root_mean_squared_error)
	1.95s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-0.0378	 = Validation score   (-root_mean_squared_error)
	0.27s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting wit

...

	-0.0454	 = Validation score   (-root_mean_squared_error)
	8.05s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0443	 = Validation score   (-root_mean_squared_error)
	8.04s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-0.0437	 = Validation score   (-root_mean_squared_error)
	0.35s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0443	 = Validation score   (-root_mean_squared_error)
	3.49s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-0.043	 = Validation score   (-root_mean_squared_error)
	0.27s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with

[1000]	valid_set's rmse: 0.044455
[2000]	valid_set's rmse: 0.044358
[3000]	valid_set's rmse: 0.0442732


	-0.0394	 = Validation score   (-root_mean_squared_error)
	13.1s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0393	 = Validation score   (-root_mean_squared_error)
	7.56s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-0.038	 = Validation score   (-root_mean_squared_error)
	0.34s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0395	 = Validation score   (-root_mean_squared_error)
	1.87s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-0.0379	 = Validation score   (-root_mean_squared_error)
	0.27s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with

[1000]	valid_set's rmse: 0.0389202


	-0.043	 = Validation score   (-root_mean_squared_error)
	7.79s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0423	 = Validation score   (-root_mean_squared_error)
	6.51s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-0.0406	 = Validation score   (-root_mean_squared_error)
	0.33s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0429	 = Validation score   (-root_mean_squared_error)
	1.47s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-0.0401	 = Validation score   (-root_mean_squared_error)
	0.27s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 3 child models (S1F1 - S1F3) | Fitting with

In [8]:
result = pd.DataFrame(assets_final_return).T

In [11]:
L  = []
for i in range(1001,1701):
    for j in range(0,54):
        code_str='s{}d{}'.format(j,i)
        L.append(code_str)
        

df_res = pd.DataFrame(L, columns=['date_time'])

In [12]:
df_res['return']=result.values.flatten()
df_res=df_res.set_index('date_time')
df_res

,return
date_time,
s0d1001,0.056922
s1d1001,0.006075
s2d1001,0.085466
s3d1001,0.004244
s4d1001,0.056535
...,...
s49d1700,-0.022095
s50d1700,0.019024
s51d1700,-0.080868


In [ ]:
# df_fin=pd.merge(df_test_fundamental, df_res, on ='date_time',how ='left')
# df_fin

In [13]:
df_res.to_csv('submission.csv')